Setup imports

In [83]:
import requests
import pandas as pd
import geopandas as gpd
from keplergl import KeplerGl
import h3
import json

Query OSM for all pubs in the UK using the Overpass API
Note - Scotland commented out

In [84]:
overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = """
[out:json];
area["ISO3166-1"="GB"][admin_level=2];
node["amenity"="pub"](area);
out;
"""
response = requests.get(overpass_url, params={'data': overpass_query})
data = response.json()

# area["ISO3166-2"="GB-SCT"];

Create Geodataframe of results

In [85]:
df = pd.DataFrame.from_dict(data["elements"])
df.head()
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lat, df.lon))
gdf = gdf.set_geometry("geometry")
gdf.head()

,type,id,lat,lon,tags,geometry
0,node,262706,51.035030,-0.725178,"{'addr:postcode': 'GU27 3HA', 'addr:street': '...",POINT (51.03503 -0.72518)
1,node,262707,51.043900,-0.854808,"{'addr:postcode': 'GU33 7JB', 'addr:street': '...",POINT (51.04390 -0.85481)
2,node,262708,51.024400,-0.726258,"{'addr:city': 'Haslemere', 'addr:place': 'Henl...",POINT (51.02440 -0.72626)
3,node,262709,50.993224,-0.779150,"{'addr:place': 'Stedham', 'addr:postcode': 'GU...",POINT (50.99322 -0.77915)
4,node,262714,51.035900,-0.805651,"{'addr:city': 'Liphook', 'addr:postcode': 'GU3...",POINT (51.03590 -0.80565)


Add H3 index to the GeoDataFrame
This will allow Kepler to render a H3 layer

In [86]:
h3_level = 5
 
def geom_to_h3(row):
  return h3.geo_to_h3(row.geometry.x, row.geometry.y, h3_level)

gdf["hex_id"] = gdf.apply(geom_to_h3, axis=1)
gdf.head()

,type,id,lat,lon,tags,geometry,hex_id
0,node,262706,51.035030,-0.725178,"{'addr:postcode': 'GU27 3HA', 'addr:street': '...",POINT (51.03503 -0.72518),85194a4bfffffff
1,node,262707,51.043900,-0.854808,"{'addr:postcode': 'GU33 7JB', 'addr:street': '...",POINT (51.04390 -0.85481),85194a4bfffffff
2,node,262708,51.024400,-0.726258,"{'addr:city': 'Haslemere', 'addr:place': 'Henl...",POINT (51.02440 -0.72626),85194a4bfffffff
3,node,262709,50.993224,-0.779150,"{'addr:place': 'Stedham', 'addr:postcode': 'GU...",POINT (50.99322 -0.77915),85194a4bfffffff
4,node,262714,51.035900,-0.805651,"{'addr:city': 'Liphook', 'addr:postcode': 'GU3...",POINT (51.03590 -0.80565),85194a4bfffffff


Create a DataFrame of the H3 index and pub count
This is what will be rendered on the map

In [87]:
	
counts = gdf.groupby(["hex_id"]).size().reset_index(name='value')
counts.head()

,hex_id,value
0,8509a093fffffff,1
1,8509a097fffffff,1
2,8509a463fffffff,1
3,8509a46ffffffff,1
4,8509a473fffffff,1


Save to CSV so we can upload this dataset to https://kepler.gl/demo to play with.
Currently it's not possible to generate a config file directly through Jupyter Notebooks
https://github.com/keplergl/kepler.gl/issues/1626

In [88]:
counts.to_csv("pub_counts.csv", index=False)

Render map in notebook

In [90]:
config = json.load(open("config.json"))
map = KeplerGl(show_docs=False, config=config)
map.add_data(counts)
map

KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': '678wozt', 'type': …

Finally, save output for deployment

In [91]:
map.save_to_html(file_name="index.html")

Map saved to index.html!
